In [1]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import openpyxl
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import os
from tabula import read_pdf
import pdfplumber
from time import sleep
import pypdfium2 as pdfium
import openpyxl
import re

In [2]:
def login_commec(browser,username,password):
    browser.get("https://www2.commsec.com.au/secure/login/netbank")
    time.sleep(2)
    kw_input = browser.find_element(By.XPATH, '//*[@id="username"]')
    kw_input.click()
    time.sleep(1)
    kw_input.send_keys(username)
    time.sleep(1)
    kw_input = browser.find_element(By.XPATH, '//*[@id="password"]')
    kw_input.click()
    kw_input.send_keys(password)
    browser.find_element(By.XPATH, '//*[@id="login"]').click()
    time.sleep(2)

In [3]:
def get_last_file(download_dir):
    file_list = os.listdir(download_dir)
    file_list.sort(key=lambda file: os.path.getmtime(os.path.join(download_dir,file)))
    return file_list[-1]

In [4]:
def renamepdf(new_file_name, download_dir):
    get_last_file(download_dir)
#     new_file_name = 'Trading-Halt ('+ index +').PDF'
    os.rename(os.path.join(download_dir,get_last_file(download_dir)), os.path.join(download_dir,new_file_name))

In [5]:
def find_word(path, key_works):
    pdf = pdfium.PdfDocument(path)
#     key_works = ['placement','raising','SPP','share purchase plan', 'IPO', 'Right issue']
    get_word = 0
    for page in pdf:
        textpage = page.get_textpage()
        text_all = textpage.get_text_range()
        for key_work in key_works:
            searcher = textpage.search(key_work, match_case=False, match_whole_word=False)
            first_occurrence = searcher.get_next()
            if first_occurrence != None:
#                 print('ok')
                get_word = 1
                break
#             else:
#                 print('no')
        if get_word == 1:
            break
    return get_word

In [6]:
def check_file(file_name ,file_index, path, download_dir, last_pdf):
    file_intact = file_name.format(file_index = str(file_index))
    while file_intact not in path:
                    time.sleep(1)
                    try:
                        path1 = os.listdir(download_dir)
                        now_pdf = get_last_file(download_dir)
                        if now_pdf != last_pdf:
                            time.sleep(1)
                            renamepdf(file_intact, download_dir)
                    except Exception as error:
                        print("An exception occurred:", error)
                    if file_intact in path1:
                        break

In [7]:
def write_excel(info,excel_dir, sheet_name):
    writer = pd.ExcelWriter(excel_dir)
    data = pd.DataFrame(info)
    data.to_excel(writer, sheet_name=sheet_name, index=False, header=False)
    writer.save()
    print("xlsx write successfully！")

In [8]:
def add_to_excel(data_info, excel_dir, sheet_name):
    writer = pd.ExcelWriter(excel_dir, mode='a', engine='openpyxl',if_sheet_exists='overlay')
    df1 = pd.DataFrame(pd.read_excel(excel_dir, sheet_name=sheet_name)) 
    df_cols = df1.shape[1] 
    data = pd.DataFrame(data_info)
    data.to_excel(writer, sheet_name=sheet_name,startcol=df_cols+1, index=False, header=False)
    writer.save()
    print("xlsx write successfully！")

In [9]:
def get_company(browser):
    action = ActionChains(browser)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    products = soup.find('div',class_='dailyTable')
    products1 = products.find('tbody')
    i=0
    j=1
    index = 1
    mat = [['Company Name','Ticker','Last price','Market Cap','Last price_d','Market Cap_d']]
    for product in products1:
        ln = []
        if('flow' in product.find_all('td')[3].text.lower() or '5b' in product.find_all('td')[3].text.lower() or '4c' in product.find_all('td')[3].text.lower()):
            i=i+1
            print(i)
            action.move_to_element(browser.find_element(By.XPATH,'//*[@id="module_8"]/section/div/div/div[2]/div/div[3]/table/tbody/tr['+ str(index) +']/td[1]/a')).perform()
            time.sleep(2)
            soup1 = BeautifulSoup(browser.page_source, 'lxml').find('div',class_='dailyTable').find('tbody').find_all('tr')[index-1]
            print(soup1.find_all('td')[0].find('div').find_all('tr')[0].find_all('td')[1].text)
            print(soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.strip('$').replace(' ', ''))
            print(soup1.find_all('td')[0].find('div').find_all('div')[1].find_all('div')[1].find('div').find_all('a')[2].get('href'))

            print(product.find_all('td')[0].text,'  ',product.find_all('td')[1].text, ' ', 'https://research.commsec.com.au'+product.find_all('td')[3].find('a').get('href'), ' ')

            ln.append(product.find_all('td')[1].text)
            ln.append(product.find_all('td')[0].text)
            try:
                ln.append(soup1.find_all('td')[0].find('div').find_all('tr')[0].find_all('td')[1].text)
                ln.append('$'+soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.strip('$').replace(' ', ''))
                ln.append(float(soup1.find_all('td')[0].find('div').find_all('tr')[0].find_all('td')[1].text.replace('$', '')))
                aa = soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.replace('$', '')
                if('M' in soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.replace('$', '')):
                    aa = float(soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.replace('$', '').replace('M', ''))
                ln.append(aa)
            except Exception as error:
                print("An exception occurred:", error)
                ln.append(error)
            ln.append(soup1.find_all('td')[0].find('div').find_all('div')[1].find_all('div')[1].find('div').find_all('a')[2].get('href'))
            ln.append('https://research.commsec.com.au'+product.find_all('td')[3].find('a').get('href'))
            print(ln)
            mat.append(ln)
        index = index + 1
    return mat

In [10]:
def get_announce(excel_dir,sheet_name,browser,download_dir):
    df = pd.read_excel(excel_dir, sheet_name = sheet_name)
    key_works = ['placement','raising','spp','share purchase plan', 'ipo', 'right issue','raises']
    file_index = 1
    find_placement = 0
    total_placement = [['time', 'time_url']]
    file_name_pdf = 'Trading-Halt ({file_index}).PDF'
    wait = 0
    for i in range (0,len(df)):
        get_work = 0
        last_pdf = ''
        now_pdf = ''
        url_ann = ''
        placement = ''
        palce = []
        print(i+1)
        print(df.iloc[i][6])
        url = df.iloc[i][6]
        browser.get(url)
#         time.sleep(3)
        while wait == 0:
            wait = 1
            time.sleep(1)
            try:
                browser.find_element(By.XPATH, '/html/body/app-root/div/div/sub-nav/nav/ul/li[2]/a')
            except  Exception as error:
                print("An exception occurred:", error)
                wait = 0
        kw_input = browser.find_element(By.XPATH, '/html/body/app-root/div/div/sub-nav/nav/ul/li[2]/a')
        kw_input.click()
        time.sleep(1)
        while wait == 0:
            wait = 1
            time.sleep(1)
            try:
                browser.find_element(By.XPATH, '//*[@id="announcements-container"]/section/div[1]/div[1]/header/section/div/div/section/div/label')
            except  Exception as error:
                print("An exception occurred:", error)
                wait = 0
        kw_input = browser.find_element(By.XPATH, '//*[@id="announcements-container"]/section/div[1]/div[1]/header/section/div/div/section/div/label')
        kw_input.click()
        while wait == 0:
            wait = 1
            time.sleep(1)
            try:
                browser.find_element(By.XPATH, '//*[@id="announcements-container"]/section/div[1]/div[1]/header/section/div/div/section/div/ul/li[4]')
            except  Exception as error:
                print("An exception occurred:", error)
                wait = 0
        kw_input = browser.find_element(By.XPATH, '//*[@id="announcements-container"]/section/div[1]/div[1]/header/section/div/div/section/div/ul/li[4]')
        kw_input.click()
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source, 'lxml')
        table_ann = soup.find('table',class_='table table-primary')
        announcements = table_ann.find('tbody').find_all('tr')
        for product in announcements:
            get_broker = 0
            description = product.find_all('td')[2].text.strip()
            for word in key_works:
                if word in description.lower():
                    find_placement = 6
                    placement = product.find_all('td')[2].find('a').get('href')
                    break
            find_placement = find_placement - 1
            if 'trading halt'in description.lower():
                print(product.find_all('td')[0].text)
                p_time = product.find_all('td')[0].text
                url_ann = product.find_all('td')[2].find('a').get('href')
                try:
                    last_pdf = get_last_file(download_dir)
                except Exception as error:
                    print("An exception occurred:", error)
                browser.get(url_ann)
                path = os.listdir(download_dir)
                check_file(file_name_pdf ,file_index, path, download_dir, last_pdf)
                pdf_name = file_name_pdf.format(file_index = str(file_index))
                pdf_path = os.path.join(download_dir, pdf_name)
                print(pdf_path)
                file_index = file_index + 1
                get_work = find_word(pdf_path,key_works)
                if get_work == 1:
                    if find_placement > 0:
                        print(placement)
                        palce.append(p_time)
                        palce.append(placement)
                        total_placement.append(palce)
                        break

        if get_work == 0 or find_placement <=0:
            total_placement.append(palce)

        time.sleep(2)
    return total_placement

In [11]:
def get_cash_pdf_v2(pdf_path):
    pattern = re.compile(r"\d+,*\d+")
    pattern1 = re.compile(r"\d")
    pattern2 = re.compile(r"\d\.\d")
    cash = None
    debt = None
    print(pdf_path)
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(0,len(pdf.pages)):
            pdf_page = pdf.pages[i]
            f = 0
            page_lines = pdf_page.extract_text_lines(layout = True, return_chars=False)
            for i in page_lines:
                if 'reconciliation of cash'in i['text'].lower():
                    f = 1
                    continue
                if 'cash and cash equiv' in i['text'].lower() and f == 1:
                    print(i['text'].split())
                    print(page_lines[page_lines.index(i)+1]['text'].split())
                    if (len(pattern.findall(i['text'].split()[-1])) != 0 or len(pattern1.findall(i['text'].split()[-1])) != 0) and (len(pattern.findall(i['text'].split()[-2])) != 0 or len(pattern1.findall(i['text'].split()[-2])) != 0) and len(pattern.findall(i['text'].split()[-3])) == 0:
                        print(i['text'].split()[-2])
                        cash = i['text'].split()[-2]
                    elif (len(pattern2.findall(page_lines[page_lines.index(i)+1]['text'].split()[-1])) == 0 and len(pattern2.findall(page_lines[page_lines.index(i)+1]['text'].split()[-2])) == 0)  and (len(pattern.findall(page_lines[page_lines.index(i)+1]['text'].split()[-1])) != 0 or len(pattern1.findall(page_lines[page_lines.index(i)+1]['text'].split()[-1])) != 0) and (len(pattern.findall(page_lines[page_lines.index(i)+1]['text'].split()[-2])) != 0 or len(pattern1.findall(page_lines[page_lines.index(i)+1]['text'].split()[-2])) != 0):
                        print(page_lines[page_lines.index(i)+1]['text'].split()[-2])
                        cash = page_lines[page_lines.index(i)+1]['text'].split()[-2]
                    else:
                        print(None)
                        cash = None
                    f = 0
                if 'total financing'in i['text'].lower():
                    print(i['text'].lower().split())
                    if i['text'].lower().split()[-2] != 'facilities' and i['text'].lower().split()[-2] != 'financing' and i['text'].lower().split()[-2] != 'total':
                        print(i['text'].lower().split()[-2])
                        debt = i['text'].lower().split()[-2]
                    elif i['text'].lower().split()[-1] == 'facilities':
                        print(' ')
                        debt = ' '
                    else:
                        print(None)
                        debt = None
    return cash, debt

In [12]:
def get_cash_debt(path):
    inside1 = []
    with pdfplumber.open(path) as pdf:
        cash = None
        debt = None
        for page in pdf.pages:
            tables = page.extract_tables()
            try:
                for table in tables:
                    if 'Reconciliation' in table[0][0]: 
                        print(table[-1][-2])
                        cash = table[-1][-2]
                    if 'Financing facilities' in table[0][0]: 
                        if(table[4][2] == '-' or table[4][2] == '' or table[4][2] == ' ' or table[4][2] == '–' or table[4][2] == 'Nil' ):
                            debt = 0
                        elif(table[4][2] != None):
                            debt = table[4][2]
                        print(table[4][2])  
            except Exception as error:
                print("An exception occurred:", error)
        print('cash '+str(cash).replace(',', ''))
        print('debt '+str(debt).replace(',', ''))
        debt = str(debt).replace(',', '')
        try:
            cash = str(cash).replace(',', '')
            cash = float(cash)/1000
        except Exception as error:
                print("An exception occurred:", error)
                cash = None
        try:
            if '-' not in debt:
                debt = float(debt)/1000
        except Exception as error:
                print("An exception occurred:", error)
                debt = None
        if cash == None or debt == None:
            print(1)
            cash1, debt1 = get_cash_pdf_v2(path)
        if cash == None:
            cash = cash1
            try:
                cash = str(cash).replace(',', '')
                cash = float(cash)/1000
            except Exception as error:
                print("An exception occurred:", error)
                cash = None
        if debt == None:
            debt = debt1
            if(debt == '-' or debt == '' or debt == ' ' or debt == '–' or debt.lower() == 'nil' ):
                debt = 0
            debt = str(debt).replace(',', '')
            try:
                if '-' not in debt:
                    debt = float(debt)/1000
            except Exception as error:
                print("An exception occurred:", error)
                debt = None
        debt = str(debt)
        cash = str(cash)
        print(cash)
        print(debt)
        inside1.append(cash)
        inside1.append(debt)
        cash = '$'+str(cash)+'M'
        inside1.append(cash)
        print(inside1)
    return inside1

In [13]:
def catch_cash(excel_dir,sheet_name, browser, download_dir):
    df = pd.read_excel(excel_dir,sheet_name = sheet_name)
    cash_flow_pdf = 'Cash_Flow ({file_index}).PDF'
    total_list = [['cash','debt','cash']]
    file_index = 1
    last_pdf = ''
    for i in range (0,len(df)):
        print(i+1)
        print(df.iloc[i][7])
        url = df.iloc[i][7]
        try:
            last_pdf = get_last_file(download_dir)
        except Exception as error:
            print("An exception occurred:", error)
        browser.get(url)
        path = os.listdir(download_dir)
        check_file(cash_flow_pdf ,file_index, path, download_dir, last_pdf)
        pdf_name = cash_flow_pdf.format(file_index = str(file_index))
        pdf_path = os.path.join(download_dir, pdf_name)
        print(pdf_path)
        file_index = file_index + 1
        cashlist = get_cash_debt(pdf_path)
        total_list.append(cashlist)
    return total_list

In [14]:
def check_info_company(company_info):
    ch = 0
    for i in company_info:
        if len(i)!=8:
            print(i)
            ch = 1
    if ch == 0:
        print('no data lost')

open the web

In [16]:
chrome_options = Options()


download_dir = 'C:\\Users\\N\\Desktop'
file_name = '0725_02'
chromedriver_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver"
password = 'Nc139687@NC1' #commec password
username = '78531231'   #commec username
excel_downlaod_dir = "C:\\Users\\N\\Desktop"
excel_name = '0725.xlsx'  #excel file name
excel_dir = os.path.join(excel_downlaod_dir, excel_name)
sheet_name = '01'   #excel sheetname
target_url = "https://www2.commsec.com.au/research/tools"


download_dir = os.path.join(download_dir, file_name)

os.makedirs(download_dir, exist_ok=True)

chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir, 
    "download.prompt_for_download": False, 
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  
})

# service = Service(executable_path = chromedriver_path)
service = Service()
options = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=service, options=chrome_options)
time.sleep(3)

login_commec(browser,username,password)

browser.get(target_url)
time.sleep(2)
browser.find_element(By.XPATH, '//*[@id="module_2"]/ul/li[4]').click()
time.sleep(5)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00007FF6C4589412+29090]
	(No symbol) [0x00007FF6C44FE239]
	(No symbol) [0x00007FF6C43BB1DA]
	(No symbol) [0x00007FF6C438FAF5]
	(No symbol) [0x00007FF6C443E2C7]
	(No symbol) [0x00007FF6C4455EB1]
	(No symbol) [0x00007FF6C4436493]
	(No symbol) [0x00007FF6C44009D1]
	(No symbol) [0x00007FF6C4401B31]
	GetHandleVerifier [0x00007FF6C48A871D+3302573]
	GetHandleVerifier [0x00007FF6C48F4243+3612627]
	GetHandleVerifier [0x00007FF6C48EA417+3572135]
	GetHandleVerifier [0x00007FF6C4645EB6+801862]
	(No symbol) [0x00007FF6C450945F]
	(No symbol) [0x00007FF6C4504FB4]
	(No symbol) [0x00007FF6C4505140]
	(No symbol) [0x00007FF6C44F461F]
	BaseThreadInitThunk [0x00007FF92468257D+29]
	RtlUserThreadStart [0x00007FF92520AF28+40]


Loading the page, interrupt this program after the page finalize the loading

In [18]:
while(True):
    time.sleep(2)
    browser.execute_script("window.scrollBy(0,document.body.scrollHeight)")

KeyboardInterrupt: 

catch info, load into excel

In [33]:
company_info = get_company(browser)

1
$0.120
3.86M
https://www2.commsec.com.au/quotes/?stockCode=BUS
BUS    Bubalus Resources Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzE1NDMtNFVKRUE4MkxWM0ZIVkdCSFNSU0tRQlZDSEUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Bubalus Resources Limited', 'BUS', '$0.120', '$3.86M', 0.12, 3.86, 'https://www2.commsec.com.au/quotes/?stockCode=BUS', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzE1NDMtNFVKRUE4MkxWM0ZIVkdCSFNSU0tRQlZDSEUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
2
$0.019
13.20M
https://www2.commsec.com.au/quotes/?stockCode=ACP
ACP    Audalia Resources Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzE1MzgtME5FU1RNSjZGVFVBQzEzSVY2TkZISTVPNUEiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Audalia Resources Limited', 'ACP', '$0.019', '$13.20M', 0.019, 13.2, 'https://www2.commsec.com.au/quotes/?stockCode=ACP', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzE1MzgtME5FU1RNSjZGVF

$0.022
13.40M
https://www2.commsec.com.au/quotes/?stockCode=AT1
AT1    Atomo Diagnostics Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzEzMDMtN0JUNUQxSTNHN0pBNDI4TEtNNDY1MUNJMjEiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Atomo Diagnostics Limited', 'AT1', '$0.022', '$13.40M', 0.022, 13.4, 'https://www2.commsec.com.au/quotes/?stockCode=AT1', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzEzMDMtN0JUNUQxSTNHN0pBNDI4TEtNNDY1MUNJMjEiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
18
$0.295
81.70M
https://www2.commsec.com.au/quotes/?stockCode=SEG
SEG    Sports Entertainment Group Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzEyODEtM0FENlNSTjJQT0lJNExBSE4xU1RTMUQ5UkYiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Sports Entertainment Group Limited', 'SEG', '$0.295', '$81.70M', 0.295, 81.7, 'https://www2.commsec.com.au/quotes/?stockCode=SEG', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4Mz

$0.070
28.40M
https://www2.commsec.com.au/quotes/?stockCode=MPA
MPA    Mad Paws Holdings Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExOTgtMUQxVjcwVEtHSzQxTFVLODBGTFRBUko2TEIiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Mad Paws Holdings Limited', 'MPA', '$0.070', '$28.40M', 0.07, 28.4, 'https://www2.commsec.com.au/quotes/?stockCode=MPA', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExOTgtMUQxVjcwVEtHSzQxTFVLODBGTFRBUko2TEIiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
34
$0.103
54.30M
https://www2.commsec.com.au/quotes/?stockCode=PSC
PSC    Prospect Resources Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExODgtNFBJQzJONjQ5OTFCUTZMRTNIQ0U2VjlTOTkiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Prospect Resources Limited', 'PSC', '$0.103', '$54.30M', 0.103, 54.3, 'https://www2.commsec.com.au/quotes/?stockCode=PSC', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExODgtNFBJQzJONjQ

$0.053
5.08M
https://www2.commsec.com.au/quotes/?stockCode=ERW
ERW    Errawarra Resources Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExMzEtMkk1NUpLM0lHM01PMVZONU41SERWUkcwVlMiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Errawarra Resources Ltd', 'ERW', '$0.053', '$5.08M', 0.053, 5.08, 'https://www2.commsec.com.au/quotes/?stockCode=ERW', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExMzEtMkk1NUpLM0lHM01PMVZONU41SERWUkcwVlMiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
50
$0.205
88.20M
https://www2.commsec.com.au/quotes/?stockCode=DEV
DEV    DevEx Resources Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExMjgtMEdMRVMzUTFITFQyM1FKTUszS0cxTFBOVksiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['DevEx Resources Limited', 'DEV', '$0.205', '$88.20M', 0.205, 88.2, 'https://www2.commsec.com.au/quotes/?stockCode=DEV', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzExMjgtMEdMRVMzUTFITFQyM1FKTU

In [35]:
check_info_company(company_info)

['Company Name', 'Ticker', 'Last price', 'Market Cap', 'Last price_d', 'Market Cap_d']


In [36]:
write_excel(company_info,excel_dir, sheet_name)

xlsx write successfully！


In [50]:
announce_info = get_announce(excel_dir,sheet_name, browser, download_dir)

1
https://www2.commsec.com.au/quotes/?stockCode=BUS
14/09/2023
An exception occurred: list index out of range
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\0725_02\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\0725_02\\Trading-Halt (1).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\0725_02\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\0725_02\\Trading-Halt (1).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\0725_02\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\0725_02\\Trading-Halt (1).PDF'
C:\Users\N\Desktop\0725_02\Trading-Halt (1).PDF


D:\anaconda3\lib\site-packages\pypdfium2\_helpers\textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


2
https://www2.commsec.com.au/quotes/?stockCode=ACP
3
https://www2.commsec.com.au/quotes/?stockCode=UVA
02/05/2024
C:\Users\N\Desktop\0725_02\Trading-Halt (2).PDF
4
https://www2.commsec.com.au/quotes/?stockCode=LLL


KeyboardInterrupt: 

In [39]:
add_to_excel(announce_info, excel_dir, sheet_name)

xlsx write successfully！


In [51]:
cash_info = catch_cash(excel_dir,sheet_name, browser, download_dir)

1
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzE1NDMtNFVKRUE4MkxWM0ZIVkdCSFNSU0tRQlZDSEUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\0725_02\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\0725_02\\Cash_Flow (1).PDF'
C:\Users\N\Desktop\0725_02\Cash_Flow (1).PDF
2,898
-
cash 2898
debt 0
2.898
0.0
['2.898', '0.0', '$2.898M']
2
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzE1MzgtME5FU1RNSjZGVFVBQzEzSVY2TkZISTVPNUEiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\0725_02\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\0725_02\\Cash_Flow (2).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\0725_02\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\0725_02\\Cash_Flow (2).PDF'
C:\Users\N\Desktop\0725_02\Cash_Flow (2).PDF
An exception occurred: argumen

11.988
0.0
['11.988', '0.0', '$11.988M']
17
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzEzMDMtN0JUNUQxSTNHN0pBNDI4TEtNNDY1MUNJMjEiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\0725_02\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\0725_02\\Cash_Flow (17).PDF'
C:\Users\N\Desktop\0725_02\Cash_Flow (17).PDF
3 ,688
cash 3 688
debt None
An exception occurred: could not convert string to float: '3 688'
An exception occurred: could not convert string to float: 'None'
1
C:\Users\N\Desktop\0725_02\Cash_Flow (17).PDF
['Cash', 'and', 'cash', 'equivalents', 'at', 'end', 'of', 'quarter']
['5.5', '3', ',688', '5', ',208']
5
['7.4', 'total', 'financing', 'facilities', '-', '-']
-
0.005
0.0
['0.005', '0.0', '$0.005M']
18
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzEyODEtM0FENlNSTjJQT0lJNExBSE4xU1RTMUQ5UkYiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinEr

KeyboardInterrupt: 

In [37]:
add_to_excel(cash_info, excel_dir, sheet_name)

xlsx write successfully！


In [ ]:
os._exit(0)